In [1]:
import numpy as np
import sklearn
import matplotlib.pyplot as plt
from sklearn.decomposition import NMF 
import scipy.stats
import scipy.spatial as sp
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.metrics import pairwise_distances
from sklearn.metrics import mean_absolute_error
import matplotlib.pyplot as plt
import scipy.stats
import matplotlib.pyplot as plt

In [2]:
import tensorflow as tf
from keras.models import Model, load_model
from keras.layers import Input, Dense, Layer, InputSpec
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras import regularizers, activations, initializers, constraints, Sequential
from keras import backend as K
from keras.constraints import UnitNorm, Constraint

C:\Users\siimk\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\siimk\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\siimk\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\siimk\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passi

In [3]:
simulated_samples = np.loadtxt("simulated_samples.txt", dtype="int")
true_suitability = np.loadtxt("true_suitability.txt", dtype="float")

In [4]:
def normalize(array):
    normalized = []
    min_val = min(array)
    max_val = max(array)
    for i in range(0, len(array)):
        normalized.append((array[i] - min_val) / (max_val - min_val))
    return normalized

def compare_matrixes(generated, true, name):
    print(name)
    print('Spearman: {0}'.format(scipy.stats.spearmanr(generated, true).correlation))
    print('MAE {}'.format(mean_absolute_error(generated, true)))

In [5]:
# Simple Autoencoder
# 1 hidden layer with rank 2,3,4
# hidden layer is linear
# sigmoid activation
# binary cross-entropy
# ei kasuta biaseid

input_dim = simulated_samples.shape[1]

In [6]:
def create_and_compile_autoencoder(encoding_dim, nb_epoch, batch_size):
    autoencoder = Sequential()
    autoencoder.add(Dense(encoding_dim, activation="linear", input_shape=(input_dim,), use_bias = False))
    autoencoder.add(Dense(input_dim, activation="sigmoid", use_bias = False))
    autoencoder.compile(metrics=['accuracy'], loss='binary_crossentropy', optimizer='adadelta')
    
    autoencoder.summary()
    
    autoencoder.fit(simulated_samples, simulated_samples,
                epochs=nb_epoch,
                batch_size=batch_size,
                shuffle=True,
                verbose=0)
    
    train_predictions = autoencoder.predict(simulated_samples)
    print('Train reconstrunction error\n', sklearn.metrics.mean_squared_error(simulated_samples, train_predictions))
    
    result_flattened = normalize(train_predictions.flatten())
    true_flattened = true_suitability.flatten()
    compare_matrixes(result_flattened, true_flattened, 'first version')

In [8]:
create_and_compile_autoencoder(2, 50, 64)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 2)                 2000      
_________________________________________________________________
dense_4 (Dense)              (None, 1000)              2000      
Total params: 4,000
Trainable params: 4,000
Non-trainable params: 0
_________________________________________________________________


InternalError: GPU sync failed

In [ ]:
create_and_compile_autoencoder(2, 50, 256)

In [92]:
create_and_compile_autoencoder(3, 50, 256)

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_17 (Dense)             (None, 3)                 3000      
_________________________________________________________________
dense_18 (Dense)             (None, 1000)              3000      
Total params: 6,000
Trainable params: 6,000
Non-trainable params: 0
_________________________________________________________________
Train reconstrunction error
 0.017166374841216917
first version
Spearman: -0.07774715162145206
MAE 0.432991315605989


In [93]:
create_and_compile_autoencoder(4, 50, 256)

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_19 (Dense)             (None, 4)                 4000      
_________________________________________________________________
dense_20 (Dense)             (None, 1000)              4000      
Total params: 8,000
Trainable params: 8,000
Non-trainable params: 0
_________________________________________________________________
Train reconstrunction error
 0.01709473382894207
first version
Spearman: -0.07374129554851733
MAE 0.43306719272907146


In [94]:
create_and_compile_autoencoder(2, 100, 256)

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_21 (Dense)             (None, 2)                 2000      
_________________________________________________________________
dense_22 (Dense)             (None, 1000)              2000      
Total params: 4,000
Trainable params: 4,000
Non-trainable params: 0
_________________________________________________________________
Train reconstrunction error
 0.015012770138408047
first version
Spearman: 0.04712597214689463
MAE 0.43313156319546886


In [95]:
create_and_compile_autoencoder(3, 100, 256)

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_23 (Dense)             (None, 3)                 3000      
_________________________________________________________________
dense_24 (Dense)             (None, 1000)              3000      
Total params: 6,000
Trainable params: 6,000
Non-trainable params: 0
_________________________________________________________________
Train reconstrunction error
 0.015040062391500027
first version
Spearman: 0.0435693640637632
MAE 0.4331130550694973


In [96]:
create_and_compile_autoencoder(4, 100, 256)

Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_25 (Dense)             (None, 4)                 4000      
_________________________________________________________________
dense_26 (Dense)             (None, 1000)              4000      
Total params: 8,000
Trainable params: 8,000
Non-trainable params: 0
_________________________________________________________________
Train reconstrunction error
 0.014980767640982136
first version
Spearman: 0.04886012451681488
MAE 0.43306115353851504


In [97]:
create_and_compile_autoencoder(4, 200, 256)

Model: "sequential_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_27 (Dense)             (None, 4)                 4000      
_________________________________________________________________
dense_28 (Dense)             (None, 1000)              4000      
Total params: 8,000
Trainable params: 8,000
Non-trainable params: 0
_________________________________________________________________
Train reconstrunction error
 0.013997983360152507
first version
Spearman: 0.13108298762873263
MAE 0.42890617130093805


In [98]:
create_and_compile_autoencoder(4, 200, 128)

Model: "sequential_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_29 (Dense)             (None, 4)                 4000      
_________________________________________________________________
dense_30 (Dense)             (None, 1000)              4000      
Total params: 8,000
Trainable params: 8,000
Non-trainable params: 0
_________________________________________________________________
Train reconstrunction error
 0.013449014977337467
first version
Spearman: 0.17262316885787524
MAE 0.4254634661643507


In [99]:
create_and_compile_autoencoder(4, 300, 128)

Model: "sequential_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_31 (Dense)             (None, 4)                 4000      
_________________________________________________________________
dense_32 (Dense)             (None, 1000)              4000      
Total params: 8,000
Trainable params: 8,000
Non-trainable params: 0
_________________________________________________________________
Train reconstrunction error
 0.013225694973591563
first version
Spearman: 0.1895708339479884
MAE 0.4242017905189745


In [100]:
create_and_compile_autoencoder(4, 500, 128)

Model: "sequential_18"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_33 (Dense)             (None, 4)                 4000      
_________________________________________________________________
dense_34 (Dense)             (None, 1000)              4000      
Total params: 8,000
Trainable params: 8,000
Non-trainable params: 0
_________________________________________________________________
Train reconstrunction error
 0.013081734848854115
first version
Spearman: 0.20117367970332065
MAE 0.4233511931389937


In [101]:
create_and_compile_autoencoder(3, 1000, 128)

Model: "sequential_19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_35 (Dense)             (None, 3)                 3000      
_________________________________________________________________
dense_36 (Dense)             (None, 1000)              3000      
Total params: 6,000
Trainable params: 6,000
Non-trainable params: 0
_________________________________________________________________
Train reconstrunction error
 0.012937865760394558
first version
Spearman: 0.33500850180098407
MAE 0.42163352045841945


In [102]:
create_and_compile_autoencoder(3, 2000, 128)

Model: "sequential_20"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_37 (Dense)             (None, 3)                 3000      
_________________________________________________________________
dense_38 (Dense)             (None, 1000)              3000      
Total params: 6,000
Trainable params: 6,000
Non-trainable params: 0
_________________________________________________________________
Train reconstrunction error
 0.012892008631199818
first version
Spearman: 0.526028227380278
MAE 0.4220309896537194


In [ ]:
create_and_compile_autoencoder(3, 3000, 128)

Model: "sequential_21"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_39 (Dense)             (None, 3)                 3000      
_________________________________________________________________
dense_40 (Dense)             (None, 1000)              3000      
Total params: 6,000
Trainable params: 6,000
Non-trainable params: 0
_________________________________________________________________


In [10]:
import datetime


In [11]:
now = datetime.datetime.now()
create_and_compile_autoencoder(3, 100, 128)
later = datetime.datetime.now()
print((now - later).total_seconds())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 3)                 3000      
_________________________________________________________________
dense_4 (Dense)              (None, 1000)              3000      
Total params: 6,000
Trainable params: 6,000
Non-trainable params: 0
_________________________________________________________________



InternalError: 2 root error(s) found.
  (0) Internal: Blas GEMM launch failed : a.shape=(128, 1000), b.shape=(1000, 3), m=128, n=3, k=1000
	 [[{{node dense_3/MatMul}}]]
	 [[Mean_1/_63]]
  (1) Internal: Blas GEMM launch failed : a.shape=(128, 1000), b.shape=(1000, 3), m=128, n=3, k=1000
	 [[{{node dense_3/MatMul}}]]
0 successful operations.
0 derived errors ignored.

In [ ]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]=""

In [ ]:
now = datetime.datetime.now()
create_and_compile_autoencoder(3, 100, 128)
later = datetime.datetime.now()
print((now - later).total_seconds())